<a href="https://colab.research.google.com/github/TomoharuKurosu/Tomoharu_DS2/blob/main/boruta_suc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -c conda-forge boruta_py


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'conda-forge'


In [17]:
!pip install rdkit
!pip install dcekit
!pip install boruta

In [19]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.figure as figure
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from dcekit.variable_selection import search_high_rate_of_same_values, search_highly_correlated_variables
from boruta import BorutaPy



In [21]:
#データの読み込み
suppl = Chem.SDMolSupplier('logSdataset1290_2d.sdf')
mols = [mol for mol in suppl if mol is not None]

#RDkit記述子の計算
descriptor_names = [descriptor_name[0] for descriptor_name in Chem.Descriptors._descList]
descriptor_calculation = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)
RDkit = [descriptor_calculation.CalcDescriptors(mol) for mol in mols]
df = pd.DataFrame(RDkit, columns = descriptor_names)
df.drop('MolLogP',axis=1,inplace=True)

#溶解度(logS)の値を取得
df['logS'] =  [mol.GetProp('logS') if 'logS' in mol.GetPropNames() else 'None' for mol in mols]

#ここから変数選択
df = df.astype(float)

#欠損のある変数を削除
df.dropna(axis=1,inplace=True)

#分散が０の変数削除
del_num1 = np.where(df.var() == 0)
df = df.drop(df.columns[del_num1], axis=1)

#変数選択（互いに相関関係にある変数の内一方を削除）
threshold_of_r = 0.95 #変数選択するときの相関係数の絶対値の閾値
corr_var = search_highly_correlated_variables(df, threshold_of_r)
df.drop(df.columns[corr_var], axis=1, inplace=True)

#同じ値を多くもつ変数の削除
inner_fold_number = 2 #CVでの分割数（予定）
rate_of_same_value = []
threshold_of_rate_of_same_value = (inner_fold_number-1)/inner_fold_number
for col in df.columns:
    same_value_number = df[col].value_counts()
    rate_of_same_value.append(float(same_value_number[same_value_number.index[0]] / df.shape[0]))
del_var_num = np.where(np.array(rate_of_same_value) >= threshold_of_rate_of_same_value)
df.drop(df.columns[del_var_num], axis=1, inplace=True)
print(df.shape)
# => (1290, 64)


[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerator
[12:31:16] DEPRECATION WARNING: please use MorganGenerat

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
(1290, 66)


In [22]:
#訓練・テストデータの分割と標準化
df_X = df.drop(['logS'],axis=1)
df_y = df.loc[:,['logS']]
X = df_X.values
y = df_y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

#StandardScalerを使って標準化
scaler_X = StandardScaler()
scaler_y = StandardScaler()
scaled_X_train = scaler_X.fit_transform(X_train)
scaled_y_train = scaler_y.fit_transform(y_train)
scaled_X_test = scaler_X.transform(X_test)
scaled_y_test = scaler_y.transform(y_test)

#Borutaによる変数選択
#pパーセンタイルの最適化（金子研の方法）
corr_list = []
for n in range(10000):
    shadow_features = np.random.rand(scaled_X_train.shape[0]).T
    corr = np.corrcoef(scaled_X_train, shadow_features,rowvar=False)[-1]
    corr = abs(corr[corr < 0.95])
    corr_list.append(corr.max())

corr_array = np.array(corr_list)
perc = 100 * (1-corr_array.max())

#Borutaの実施
feat_selector = BorutaPy(rf(),
n_estimators='auto',  # 特徴量の数に比例して、木の本数を増やす
verbose=0,
alpha=0.05, # 有意水準
max_iter=100, # 試行回数
perc = perc, #ランダム生成変数の重要度の何％を基準とするか
random_state=0
)
feat_selector.fit(scaled_X_train, scaled_y_train)
scaled_X_train_br = scaled_X_train[:,feat_selector.support_]
scaled_X_test_br = scaled_X_test[:,feat_selector.support_]
print('pパーセンタイル:',round(perc,2))
print('boruta後の変数の数:',scaled_X_train_br.shape[1])
#"""output
#pパーセンタイル:80.79
#boruta後の変数の数: 43
#"""


pパーセンタイル: 80.78
boruta後の変数の数: 45


'output\npパーセンタイル:80.79\nboruta後の変数の数: 43\n'

In [28]:
scaled_pred_y_br.shape

(645,)

In [29]:
model = rf()
model_br = rf()
model.fit(scaled_X_train, scaled_y_train)
model_br.fit(scaled_X_train_br, scaled_y_train)
scaled_pred_y_br = np.ndarray.flatten(model_br.predict(scaled_X_test_br))
pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br)
scaled_pred_y = np.ndarray.flatten(model.predict(scaled_X_test))
pred_y = scaler_y.inverse_transform(scaled_pred_y)

print('r2_br:{}'.format(round(r2_score(y_test, pred_y_test_br),3)))
print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test_br)),3)))
print('r2:{}'.format(round(r2_score(y_test, pred_y_test),3)))
print('RMSE:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test)),3)))
#"""output
#r2_br:0.883
#RMSE_br:0.697
#r2:0.883
#RMSE:0.699


ValueError: Expected 2D array, got 1D array instead:
array=[-0.76762876  0.00448848  0.25122362  1.05658851  0.39539571  0.29565277
 -0.9162784   1.12852613  0.15830832  0.41612602 -0.72433756  1.25538174
  0.06712467  0.54372376 -0.41951798  0.59577216 -1.1567054  -1.91209987
 -0.26713293  0.38065197  0.0434753   1.20654926 -0.62602941  0.25295527
  1.3884218   1.14381411 -0.79533514  0.26552209 -0.04746097 -1.0672534
  1.55109779 -0.03786269 -0.12677046  0.33899346  0.25468692 -2.03351315
 -0.86702538 -1.00867421  0.10952532  0.97594318  0.30371731  0.79416958
  1.10997276 -0.93173955  0.67062884 -0.70449781  0.29441588 -0.86930126
  0.23044384  0.50439061 -0.38577557 -0.16224452  1.49573452  0.26705584
 -0.31888447 -0.22765135  0.60754737  0.11946993  0.79501067  0.5857286
 -2.90903453  0.06717415 -1.70355994 -0.34995519 -0.52109152 -0.39774869
 -2.46781055 -0.03875325  0.94135969  0.50557802  0.37911822 -0.04765887
  0.39128923 -1.09629561  0.93507628  0.7620104   0.47297356  0.076

In [30]:
#model_br.fit(scaled_X_train_br, scaled_y_train)
#scaled_pred_y_br = np.ndarray.flatten(model_br.predict(scaled_X_test_br)) # This line flattens the array to 1D
#pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br)

model_br.fit(scaled_X_train_br, scaled_y_train)
scaled_pred_y_br = model_br.predict(scaled_X_test_br) # Remove the flattening operation
pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br)
#scaled_pred_y = np.ndarray.flatten(model.predict(scaled_X_test))
#pred_y = scaler_y.inverse_transform(scaled_pred_y)

#print('r2_br:{}'.format(round(r2_score(y_test, pred_y_test_br),3))) # pred_y_test_br is not defined
#print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test_br)),3))) # pred_y_test_br is not defined
#print('r2:{}'.format(round(r2_score(y_test, pred_y_test),3))) # pred_y_test is not defined
#print('RMSE:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test)),3))) # pred_y_test is not defined

print('r2_br:{}'.format(round(r2_score(y_test, pred_y_br),3))) # Use the correct variable name
print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_br)),3))) # Use the correct variable name

ValueError: Expected 2D array, got 1D array instead:
array=[-6.89432474e-01  7.45954970e-02  2.64186249e-01  1.11516770e+00
  2.77544679e-01  3.23062291e-01 -9.06102906e-01  1.13169257e+00
  1.29414535e-01  4.35866809e-01 -7.35222202e-01  1.20664821e+00
  1.65878101e-01  4.45069283e-01 -4.10463931e-01  5.30513758e-01
 -1.10891191e+00 -1.89606975e+00 -2.91821285e-01  3.65908218e-01
 -5.55508174e-03  1.27720051e+00 -6.16233226e-01  2.02242715e-01
  1.43141616e+00  1.12580497e+00 -8.69598111e-01  1.80671325e-01
 -8.46666715e-02 -1.13830045e+00  1.54679341e+00 -9.09500811e-02
 -1.16430049e-01  3.80651966e-01  3.19598995e-01 -2.07017462e+00
 -8.65194777e-01 -1.16338461e+00  1.23625882e-01  9.64266921e-01
  3.04063635e-01  7.90063102e-01  1.07657668e+00 -8.37834733e-01
  5.77466165e-01 -6.77434625e-01  3.60812224e-01 -9.69885286e-01
  1.55389259e-01  4.44376624e-01 -3.17251777e-01 -2.24089098e-01
  1.53002116e+00  3.35629110e-01 -3.71773961e-01 -1.79462047e-01
  6.25803891e-01  2.21512912e-0

In [31]:
#model_br.fit(scaled_X_train_br, scaled_y_train)
#scaled_pred_y_br = np.ndarray.flatten(model_br.predict(scaled_X_test_br)) # This line flattens the array to 1D
#pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br)

model_br.fit(scaled_X_train_br, scaled_y_train)
scaled_pred_y_br = model_br.predict(scaled_X_test_br) # Remove the flattening operation
pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br.reshape(-1, 1)) # Reshape the array to 2D
#scaled_pred_y = np.ndarray.flatten(model.predict(scaled_X_test))
#pred_y = scaler_y.inverse_transform(scaled_pred_y)

#print('r2_br:{}'.format(round(r2_score(y_test, pred_y_test_br),3))) # pred_y_test_br is not defined
#print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test_br)),3))) # pred_y_test_br is not defined
#print('r2:{}'.format(round(r2_score(y_test, pred_y_test),3))) # pred_y_test is not defined
#print('RMSE:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test)),3)))

In [32]:
#model_br.fit(scaled_X_train_br, scaled_y_train)
#scaled_pred_y_br = np.ndarray.flatten(model_br.predict(scaled_X_test_br)) # This line flattens the array to 1D
#pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br)

model_br.fit(scaled_X_train_br, scaled_y_train)
scaled_pred_y_br = model_br.predict(scaled_X_test_br) # Remove the flattening operation
pred_y_br = scaler_y.inverse_transform(scaled_pred_y_br.reshape(-1, 1)) # Reshape the array to 2D
#scaled_pred_y = np.ndarray.flatten(model.predict(scaled_X_test))
#pred_y = scaler_y.inverse_transform(scaled_pred_y)

#print('r2_br:{}'.format(round(r2_score(y_test, pred_y_test_br),3))) # pred_y_test_br is not defined
#print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test_br)),3))) # pred_y_test_br is not defined
#print('r2:{}'.format(round(r2_score(y_test, pred_y_test),3))) # pred_y_test is not defined
#print('RMSE:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_test)),3))) # pred_y_test is not defined

print('r2_br:{}'.format(round(r2_score(y_test, pred_y_br),3))) # Use the correct variable name
print('RMSE_br:{}'.format(round(np.sqrt(mean_squared_error(y_test, pred_y_br)),3))) # Use the correct variable name

r2_br:0.884
RMSE_br:0.695
